In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
from src.modules import conf, fit, spec

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
data = fit.load_pcls(
        conf["Athlete"]["name"],
        conf["Athlete"]["activity_type"],
        conf["Paths"]["pcl"],
    )

100%|██████████| 957/957 [00:02<00:00, 437.80it/s]


In [5]:
from src.heuristics import random_shooting
from src.heuristics import genetic_optimization

In [15]:
r_cross = conf['GO']['r_cross']
r_mut = conf['GO']['r_mut']

In [3]:
train_df = fit.get_train_df(data)
test_df = fit.get_test_df(data)

In [7]:
full_columns = train_df.columns

In [10]:
_,_,rmse,cols = random_shooting.random_shoot(train_df=train_df,
                          test_df=test_df,
                          hmax=5,
                          endog='enhanced_speed',
                          )

100%|██████████| 5/5 [00:28<00:00,  5.72s/it]


In [11]:
genetic_optimization.genetic_algorithm(1, r_cross,r_mut ,cols,train_df,test_df,full_columns,'enhanced_speed')

  0%|          | 0/1 [00:32<?, ?it/s]

New best: 1.837819334568845


NameError: name 'selection' is not defined

In [50]:
genetic_optimization.mutation(cols[0],r_mut,full_columns)


['sin_wind_direct',
 'log_heart_rate',
 'tan_slope_steep',
 'cadence_delayed',
 'cos_dist_diff',
 'rain',
 'diff_heart_rate',
 'tan_distance',
 'diff_enhanced_altitude',
 'temp',
 'wind_speed',
 'cos_distance',
 'cos_enhanced_altitude',
 'sin_temp',
 'cos_wind_speed',
 'tan_enhanced_altitude_delayed',
 'cos_slope_ascent',
 'distance',
 'cadence',
 'slope_steep',
 'sin_heart_rate',
 'tan_heart_rate',
 'diff_temp',
 'diff_wind_speed',
 'tan_wind_direct',
 'diff_wind_direct',
 'cos_rain',
 'tan_rain',
 'diff_rain',
 'sin_dist_diff',
 'tan_dist_diff',
 'diff_dist_diff',
 'log_cadence_delayed',
 'sin_cadence_delayed',
 'diff_cadence_delayed',
 'cos_enhanced_altitude_delayed',
 'diff_enhanced_altitude_delayed',
 'sin_slope_steep',
 'cos_slope_steep']

In [20]:
def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif

In [51]:
cols[0]

['sin_wind_direct',
 'log_heart_rate',
 'tan_slope_steep',
 'cadence_delayed',
 'cos_dist_diff',
 'rain',
 'diff_heart_rate',
 'tan_distance',
 'diff_enhanced_altitude',
 'temp',
 'wind_speed',
 'cos_distance',
 'cos_enhanced_altitude',
 'sin_temp',
 'cos_wind_speed',
 'tan_enhanced_altitude_delayed',
 'cos_slope_ascent',
 'distance',
 'cadence',
 'slope_steep',
 'sin_heart_rate',
 'tan_heart_rate',
 'diff_temp',
 'diff_wind_speed',
 'tan_wind_direct',
 'diff_wind_direct',
 'cos_rain',
 'tan_rain',
 'diff_rain',
 'sin_dist_diff',
 'tan_dist_diff',
 'diff_dist_diff',
 'log_cadence_delayed',
 'sin_cadence_delayed',
 'diff_cadence_delayed',
 'cos_enhanced_altitude_delayed',
 'diff_enhanced_altitude_delayed',
 'sin_slope_steep',
 'cos_slope_steep']